In [ ]:
import os
import json
import codecs
from Levenshtein import distance
from teds import TEDS
from tqdm import tqdm
import re

In [ ]:
result_path = "./output/tablip_post/eval/result/tablip_result.json"
with open(result_path, 'r', encoding='utf-8') as json_file:
    result_collection = json.load(json_file)

teds_all = 0
n_jobs = 32
teds = TEDS(structure_only=False, n_jobs=n_jobs)
s_teds = TEDS(structure_only=True, n_jobs=n_jobs)

results_json = {}

for img_name, item in tqdm(result_collection[0].items()):
    pred_string = item['pred_string']
    answer_string = item['answer_string']

    pred_string = "<html><body><table>" + pred_string + "</table></body></html>"
    answer_string = "<html><body><table>" + answer_string + "</table></body></html>"
    
    if img_name in results_json.keys():
        raise ValueError()
    results_json[img_name] = {}
    results_json[img_name]["pred"] = pred_string
    results_json[img_name]["gt"] = answer_string


teds_all = teds.batch_evaluate(results_json)
s_teds_all = s_teds.batch_evaluate(results_json)

teds_scores = 0
for img_name, output in teds_all.items():
    teds_scores += output['scores']

s_teds_scores = 0
for img_name, output in s_teds_all.items():
    s_teds_scores += output['scores']

teds_scores = round(teds_scores * 100 / (len(teds_all)-1), 2)
s_teds_scores = round(s_teds_scores * 100 / len(s_teds_all), 2)

for img_name, html in results_json.items():
    html["teds"] = teds_all[img_name]["scores"]
    html["s-teds"] = s_teds_all[img_name]["scores"]

new_result_path = '/'.join(result_path.split('/')[:-1]) + f"/result_TEDS_{teds_scores}_S-TEDS_{s_teds_scores}" + ".json"
with open(new_result_path, 'w', encoding='utf-8') as json_file:
    json.dump(results_json, json_file)

print(f"dataset: {result_path}\nTEDS: {teds_scores}\nS-TEDS: {s_teds_scores}")